In [0]:
# read the data
df = spark.sql("SELECT * FROM databricks_dev.silver.silver_orders")
df.display()

In [0]:
df_dimcus = spark.sql("""SELECT dim_customer_key, customer_id AS dim_customer_id 
                      FROM databricks_dev.gold.dim_customers""")

df_dimprod = spark.sql("""SELECT product_id AS dim_product_key, 
                       product_id AS dim_product_id 
                       FROM databricks_dev.gold.dim_products""")

In [0]:
df_fact = df.join(df_dimcus, df.customer_id == df_dimcus.dim_customer_id, how="left").join(df_dimprod, df.product_id == df_dimprod.dim_product_id, how="left")

df_fact_new = df_fact.drop("dim_customer_id", "dim_product_id", "ingest_date", "customer_id", "product_id")

In [0]:
df_fact_new.display()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_dev.gold.fact_orders"):
    dlt_obj = DeltaTable.forName(spark, "databricks_dev.gold.fact_orders")
    dlt_obj.alias("trg").merge(df_fact_new.alias("src"), "trg.order_id = src.order_id AND trg.dim_customer_key = src.dim_customer_key AND trg.dim_product_key = src.dim_product_key") \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()
else:
    df_fact_new.write.format("delta")\
        .option("path", "abfss://gold@databricksdevetl.dfs.core.windows.net/fact_orders")\
        .saveAsTable("databricks_dev.gold.fact_orders")


In [0]:
%sql
select * from databricks_dev.gold.fact_orders